In [66]:
import selenium
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import lxml
import requests
from lxml import html
import math
import os
import sys
import urllib
import shutil
import pandas as pd
import cv2

In [11]:
JS_DROP_FILE = """
    var target = arguments[0],
        offsetX = arguments[1],
        offsetY = arguments[2],
        document = target.ownerDocument || document,
        window = document.defaultView || window;

    var input = document.createElement('INPUT');
    input.type = 'file';
    input.onchange = function () {
      var rect = target.getBoundingClientRect(),
          x = rect.left + (offsetX || (rect.width >> 1)),
          y = rect.top + (offsetY || (rect.height >> 1)),
          dataTransfer = { files: this.files };

      ['dragenter', 'dragover', 'drop'].forEach(function (name) {
        var evt = document.createEvent('MouseEvent');
        evt.initMouseEvent(name, !0, !0, window, 0, 0, 0, x, y, !1, !1, !1, !1, 0, null);
        evt.dataTransfer = dataTransfer;
        target.dispatchEvent(evt);
      });

      setTimeout(function () { document.body.removeChild(input); }, 25);
    };
    document.body.appendChild(input);
    return input;
"""

def drag_and_drop_file(drop_target, path):
    driver = drop_target.parent
    file_input = driver.execute_script(JS_DROP_FILE, drop_target, 0, 0)
    file_input.send_keys(path)

In [12]:
def colorScore(img1, img2):
    abs_sum_error=0
    abs_sum_error+=math.fabs(img1.T[0].mean()-img2.T[0].mean())
    abs_sum_error+=math.fabs(img1.T[1].mean()-img2.T[1].mean())
    abs_sum_error+=math.fabs(img1.T[2].mean()-img2.T[2].mean())
    return abs_sum_error

def keyPointScore(img1,img2):
    orb = cv2.ORB_create() 
    queryKeypoints, queryDescriptors = orb.detectAndCompute(img1,None) 
    trainKeypoints, trainDescriptors = orb.detectAndCompute(img2,None) 
    matcher = cv2.BFMatcher() 
    matches = matcher.match(queryDescriptors,trainDescriptors) 
    sums=0.0
    for match in matches:
        sums+=match.distance
    return sums


In [13]:
def getInformation(url:str):
    
    # Getting the webpage, creating a Response object.
    response = requests.get(url)
    
    # Extracting the source code of the page.
    data = response.text
    
    # Passing the source code to BeautifulSoup to create a BeautifulSoup object for it.
    soup = BeautifulSoup(data, 'html.parser')
        
    caption = soup.findAll("div", {"class": "asset-description__caption"})
    tags = soup.findAll("li", {"class": "asset-keywords-list__item"})
    imgur=soup.findAll("img", {"class": "asset-card__image"})
    caption_str=""
    try:
        caption_str:str=caption[0].getText()
    except:
        pass
    tag_list:str=",".join([tag.getText().replace(",","") for tag in tags])
    img_link=imgur[0]["src"]
    dictionary:{}={}
    dictionary["img_url"]=img_link
    dictionary["tags"]=tag_list
    dictionary["caption"]=caption_str
    return  dictionary



In [14]:
def saveImg(dir,original_img, name, urlLink):
        if not os.path.exists(dir):
            os.makedirs(dir) 
            
        subpath:str= os.path.join(dir,original_img)
        if not os.path.exists(subpath):
            os.makedirs(subpath) 
            
        file_name_path:str=os.path.join(subpath,name)+".jpg"
        
        img_url:str=urlLink.replace("?s=2048x2048","") #+".jpg"  

        urllib.request.urlretrieve(img_url, file_name_path)

In [108]:
def upload_file(driver, abs_img_path,url = "https://www.gettyimages.co.uk/"):
    driver.get(url)
    bah=driver.find_element_by_id('btnImageSearch').click()
    time.sleep(5)
    el=driver.find_element_by_css_selector("[ngf-drop^='dragDropUpload']")
    path = abs_img_path
    drag_and_drop_file(el,path)
    
    return driver

In [109]:
def save_all(project_path, abs_img_path, img_num, original_img, linkInfo):
    projectPath=project_path
    elementName=img_num #the original png
    img_dir_path = os.path.join(projectPath, elementName)
    if not os.path.isdir(img_dir_path):
        os.mkdir(img_dir_path)
    shutil.copy(abs_img_path, img_dir_path)
    
    counter = 1
    for link in linkInfo:
        link["src"]=elementName
        link["id"]=str(counter)
        time.sleep(2)
        print("saving Image")
        saveImg(projectPath,elementName,link["id"],link["img_url"])
        counter += 1

    print ("saving Misc Info as a clean tsv file")

    all_info=pd.DataFrame(linkInfo)
    
    return all_info

In [110]:
def create_features(project_path, img_num, original_img, all_info, target_imgs):
    src_img = cv2.imread(project_path + '/' + img_num + '/' + original_img, cv2.IMREAD_COLOR)# trainImage
    color_scores=[]
    key_point_scores=[]
    for img_name in target_imgs:
        img1 = cv2.imread(project_path + '/' + img_num + '/'+ img_name + '.jpg', cv2.IMREAD_COLOR)# queryImage

        img1=cv2.resize(img1,(500,500))
        img2=cv2.resize(src_img,(500,500))

        color_score=colorScore(img1,img2)
        key_point_score=keyPointScore(img1,img2)
        color_scores.append(color_score)
        key_point_scores.append(key_point_score)

    all_info['Color_Score'] = color_scores
    all_info['Key_Point_Score'] = key_point_scores
    all_info.to_csv(os.path.join(img_dir_path,"related_info_{}.tsv".format(img_num)),sep="\t")

In [111]:
def run_selenium(driver, abs_img_path, original_img, project_path, img_num, max_num = 6):
    driver = upload_file(driver, abs_img_path)
    time.sleep(10)
    
    topN_images=driver.find_elements_by_class_name("gallery-mosaic-asset__link")

    if len(topN_images)<max_num: 
        max_num=len(topN_images)

    linkInfo=[]
    for image in topN_images[:max]:
        linkInfo.append({"url":image.get_attribute("href")})
        
    for link in linkInfo:
        url=link["url"]
        print(url + '\n')
        time.sleep(3)
        link.update(getInformation(url))
        print("Next Link")
    
    all_info = save_all(project_path, abs_img_path, img_num, original_img, linkInfo)
    
    all_info.head()
    
    target_imgs = []
    for link in linkInfo:
        target_imgs.append(str(link['id']))
        
    create_features(project_path, img_num, original_img, all_info, target_imgs)

**RUN COLLECTION ON A SINGLE EXAMPLE**

In [112]:
driver = webdriver.Chrome('/Users/nissani/Desktop/chromedriver') #use own driver path
for el in ['01235.png', '01236.png']:
    run_selenium(driver = driver, 
                 abs_img_path = '/Users/nissani/Desktop/Hateful_Memes_Project/ExampleProject/data/img/' + el,
                 original_img = el,
                 project_path = "/Users/nissani/Desktop/Hateful_Memes_Project/ExampleProject/img_compare_test",
                 img_num = el.replace('.png', ''))

https://www.gettyimages.co.uk/detail/photo/come-on-you-can-do-it-royalty-free-image/185288341

Next Link
https://www.gettyimages.co.uk/detail/photo/ecstatic-young-man-gesturing-thumbs-up-isolated-royalty-free-image/175454964

Next Link
https://www.gettyimages.co.uk/detail/photo/portrait-of-funny-mature-businessman-royalty-free-image/1163453266

Next Link
https://www.gettyimages.co.uk/detail/photo/cheerful-arab-man-walking-with-arms-raised-towards-royalty-free-image/1050607276

Next Link
https://www.gettyimages.co.uk/detail/photo/feel-fantastic-royalty-free-image/185221981

Next Link
https://www.gettyimages.co.uk/detail/photo/giving-a-thumbs-up-in-approval-royalty-free-image/1050604302

Next Link
saving Image
saving Image
saving Image
saving Image
saving Image
saving Image
saving Misc Info as a clean tsv file
saving Misc Info as a clean tsv file
